# Countermeasure tests

In [ ]:
from pyecsca.sca.target.ectester import KeypairEnum, ParameterEnum, CurveEnum, KeyEnum, KeyClassEnum, KeyBuildEnum, KeyAgreementEnum, SignatureEnum, TransformationEnum
from pyecsca.ec.params import load_params_ectester
from pyecsca.sca.target.ectester import ECTesterTargetPCSC, KeyAgreementEnum
from pyecsca.sca.target.PCSC import PCSCTarget
from smartcard.System import readers
import numpy as np
from time import sleep
from utils import *
import os

In [ ]:
rs = readers()
for reader in rs:
    if "Gemalto" in reader.name:
        break
target = ECTesterTargetPCSC(reader)

In [ ]:
target.connect()

In [ ]:
target.select_applet()

In [ ]:
CARD = "I1"

In [ ]:
atr = target.atr.hex()
card_map = {
    "3bd518ff8191fe1fc38073c8211309": "A1",
    "3bb89600c00831fe45ffff1154305023006a": "I1",
    "3bfe1800008031fe45803180664090a5102e1083019000f2": "I2",
    "3bf81800ff8131fe454a434f507632343143": "N1",
    "3bf81300008131fe454a434f5076323431b7": "N2N9",
    "3b9495810146545601c4": "N4N10",
    "3bd518ff8191fe1fc38073c821100a": "N6",
    "3b9c9580811f039067464a01005404f272fe00c0": "F1",
    "3b90958011fe6a": "F2",
    "3b9f95803fc7a08031e073fa21106300000083f09000bb": "S1S2",
    "3bf91300008131fe454a434f503234325233a2":"N8",
    "3bf99600008131fe4553434537200e00202028":"G1"
}
print(f"{card_map[atr]} connected, {CARD} selected")
assert CARD in card_map[atr]


## Test cofactor

In [ ]:
params = load_params_ectester("tests/testcofactor/cofactor256p4_h.csv", "affine")
params_csv = read_curve_params("tests/testcofactor/cofactor256p4_h.csv")
point = get_point_bytes("tests/testcofactor/point_4n.csv")
with open("tests/testcofactor/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
params = load_params_ectester("tests/testcofactor/weakcurve.csv", "affine")
params_csv = read_curve_params("tests/testcofactor/weakcurve.csv")
point = get_point_bytes("tests/testcofactor/weakcurve_point_4n.csv")
with open("tests/testcofactor/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH_PLAIN)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

In [ ]:
result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH_PLAIN,
                         point) # pubkey as bytes
if not result.success:
    print(result)
result_lines = [serialize_ecdh_response(result,params_csv,point,key)]

In [ ]:
result.resp

In [ ]:
header = "success;error;secret[SHA1];priv;pub;curve;params;apdu;sws"
filename = f"results/{CARD}/testcofactor/ecdh_plain2.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

### Test3n - ECDH

In [ ]:
params = load_params_ectester("tests/test3n/curve_prime_gen.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve_prime_gen.csv")
point = get_point_bytes("tests/test3n/point_3n.csv")
with open("tests/test3n/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
samples = 100

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

In [ ]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point,key))

In [ ]:
target.cleanup()

In [ ]:
target.disconnect()

### Test3n - ECDSA

In [ ]:
target.connect()

In [ ]:
params = load_params_ectester("tests/test3n/curve.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve.csv")
with open("tests/test3n/key.csv") as f:
    key = int(f.read(),16)
point_bytes = get_point_bytes("tests/test3n/point_key.csv")
point = params.curve.decode_point(point_bytes)

In [ ]:
samples = 10
fixed_key = False
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
if fixed_key:
    keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
    keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
    print(target.set(KeypairEnum.KEYPAIR_LOCAL,
                 CurveEnum.external,
                 ParameterEnum.KEYPAIR,
                 keypair_dict))
else:
    print(target.generate(KeypairEnum.KEYPAIR_LOCAL))
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]

In [ ]:
result_lines = []
error = False
for _ in range(samples):
    sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if error or not sign_response.success:
        print(f"sign: {sign_response.sws}",end=", ")
        error = True
    signature = sign_response.signature
    verify_response = target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,signature,data)
    if error or not verify_response.success:
        print(f"verify: {verify_response.sws}",end=", ")
        error = True
    # verify_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    try:
        result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes, verify_response.success))
    except:
        continue

In [ ]:
result_lines

In [ ]:
filename = f"results/{CARD}/test3n/ecdsa.csv" if not fixed_key else f"results/{CARD}/test3n/ecdsa_fixed.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.cleanup()


In [ ]:
target.disconnect()

### Test3n - Keygen

In [ ]:
params = load_params_ectester("tests/test3n/curve.csv", "affine")
params_csv = read_curve_params("tests/test3n/curve.csv")

In [ ]:
samples = 1000

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
result_lines = []
for _ in range(samples):
    generate_response = target.generate(KeypairEnum.KEYPAIR_LOCAL)
    error = False
    if not generate_response.success:
        print(f"generate: {generate_response.sws}",end=", ")
        error = True
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    if not export.success:
        print(f"export: {export.sws}",end=", ")
        error = True
    
    try:
        key = int(export.params[1].hex(),16)
        point = params.curve.decode_point(export.params[0])
        point_bytes = export.params[0]
        result_lines.append(serialize_keygen_response(generate_response,key,params_csv,point_bytes))
    except:
        continue

In [ ]:
result_lines

In [ ]:
header = "success;error;priv;pub;curve;params;apdu;sws"
filename = f"results/{CARD}/test3n/keygen.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.disconnect()

### Testinverse - ECDH

In [ ]:
params = load_params_ectester("tests/testinverse/cofactor256p11_full.csv", "affine")
params_csv = read_curve_params("tests/testinverse/cofactor256p11_full.csv")
point_bytes = get_point_bytes("tests/testinverse/point_11n.csv")
with open("tests/testinverse/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
samples = 100

In [ ]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

In [ ]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point_bytes) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point_bytes,key))

In [ ]:
filename = f"results/{CARD}/testinverse/ecdh.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.cleanup()

In [ ]:
target.disconnect()

### Testinverse - ECDSA

In [ ]:
target.connect()

In [ ]:
params = load_params_ectester("tests/testinverse/cofactor256p11_full.csv", "affine")
params_csv = read_curve_params("tests/testinverse/cofactor256p11_full.csv")
point_bytes = get_point_bytes("tests/testinverse/point_11n.csv")
point = params.curve.decode_point(point_bytes)

with open("tests/testinverse/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
samples = 100
fixed_key = False
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [ ]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
if fixed_key:
    keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
    keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
    print(target.set(KeypairEnum.KEYPAIR_LOCAL,
                 CurveEnum.external,
                 ParameterEnum.KEYPAIR,
                 keypair_dict))
else:
    print(target.generate(KeypairEnum.KEYPAIR_LOCAL))
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]

In [ ]:
result_lines = []

for _ in range(samples):
    sign_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if not sign_response.success:
        print(sign_response)
    # verify_response = target.ecdsa(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes))

In [ ]:
filename = f"results/{CARD}/testinverse/ecdsa.csv" if not fixed_key else f"results/{CARD}/testinverse/ecdsa_fixedkey.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.cleanup()

In [ ]:
target.disconnect()

### Test3n - Keygen

In [ ]:
divisor = 11
params = load_params_ectester(f"tests/testinverse/cofactor256p{divisor}_full.csv", "affine")
params_csv = read_curve_params(f"tests/testinverse/cofactor256p{divisor}_full.csv")

In [ ]:
samples = 10

In [ ]:
target.allocate_ka(KeypairEnum.KEYPAIR_LOCAL)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
result_lines = []
for _ in range(samples):
    generate_response = target.generate(KeypairEnum.KEYPAIR_LOCAL)
    if not generate_response.success:
        print(generate_response)
    export = target.export(KeypairEnum.KEYPAIR_LOCAL, KeyEnum.BOTH,ParameterEnum.KEYPAIR)
    if not export.success:
        print(export)
    key = int(export.params[1].hex(),16)
    point = params.curve.decode_point(export.params[0])
    point_bytes = export.params[0]
    result_lines.append(serialize_keygen_response(generate_response,key,params_csv,point_bytes))

In [ ]:
filename = f"results/{CARD}/testinverse/keygen_{divisor}.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
target.disconnect()

### Test10 - ECDH

In [ ]:
params = load_params_ectester("tests/testk10/secp256r1.csv", "affine")
params_csv = read_curve_params("tests/testk10/secp256r1.csv")
point_bytes = get_point_bytes("tests/testk10/secgpoint.csv")
with open("tests/testk10/key_10.csv") as f:
    key = int(f.read(),16)

In [ ]:
samples = 100

In [ ]:
target.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key))

In [ ]:
result_lines = []

for _ in range(samples):
    result = target.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                         True,
                         TransformationEnum.NONE,
                         KeyAgreementEnum.ALG_EC_SVDP_DH,
                         point_bytes) # pubkey as bytes
    if not result.success:
        print(result)
    result_lines.append(serialize_ecdh_response(result,params_csv,point_bytes,key))

In [ ]:
filename = f"results/{CARD}/testk10/ecdh.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
with open(filename,"w") as f:
    for line in result_lines:
        f.write(f"{line}\n")

In [ ]:
result.params

In [ ]:
target.cleanup()


In [ ]:
target.disconnect()

### Test n+epsilon

In [ ]:
params = load_params_ectester("tests/testdn/weakcurve_32_n_x.csv", "affine")
params_csv = read_curve_params("tests/testdn/weakcurve_32_n_x.csv")
point_bytes = get_point_bytes("tests/testdn/key_point.csv")
point = params.curve.decode_point(point_bytes)

with open("tests/testdn/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
samples = 100
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

In [ ]:
result_lines = []
error = False
for _ in range(samples):
    sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
    if error or not sign_response.success:
        print(f"sign: {sign_response.sws}",end=", ")
        error = True
    signature = sign_response.signature
    try:
        result_lines.append(serialize_ecdsa_response(sign_response,data,params,key,params_csv,point_bytes, None))
    except:
        continue

In [ ]:
result_lines

In [ ]:
filename = f"results/{CARD}/testdn/ecdsa_xfixed.csv"
if os.path.isfile(filename):
    print("Measurement already exists")
header = "success;error;signature;valid;data;nonce;priv;pub;curve;params;apdu;sws"
with open(filename,"w") as f:
    f.write(f"{header}\n")
    for line in result_lines:
        f.write(f"{line}\n")

### Test small subgroup attack on verify

In [ ]:
from pyecsca.ec.mod import Mod, RawMod

In [ ]:
from pyasn1.codec.der.decoder import decode
from pyasn1.type.univ import Sequence

def parse_ecdsa_signature(signature_der):
    decoded_signature, _ = decode(signature_der, asn1Spec=Sequence())
    r = int(decoded_signature[0])
    s = int(decoded_signature[1])
    return r, s

def is_r_even(signature_der, n):
    r,s = parse_ecdsa_signature(signature_der)
    scalar = int(r*RawMod(s,n).inverse())
    return scalar%2==0 and r%2==0

In [ ]:
params = load_params_ectester("tests/verify2/cofactor256p2_h.csv", "affine")
params_csv = read_curve_params("tests/verify2/cofactor256p2_h.csv")
point_bytes = get_point_bytes("tests/verify2/key_point.csv")
point = params.curve.decode_point(point_bytes)
point_wrong_bytes = get_point_bytes("tests/verify2/key_point_wrong.csv")
point_wrong = params.curve.decode_point(point_wrong_bytes)
point_realwrong_bytes = get_point_bytes("tests/verify2/key_point_realwrong.csv")
point_realwrong = params.curve.decode_point(point_realwrong_bytes)
with open("tests/testdn/key.csv") as f:
    key = int(f.read(),16)

In [ ]:
data = bytes.fromhex("e8c86c9596e46403c211206617b8bcdd160a673b0b1304869f5559e3afe99d79")

In [ ]:
target.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
target.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [ ]:
target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetPCSC.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

In [ ]:
sign_response = target.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,True,SignatureEnum.ALG_ECDSA_SHA,data)
print(is_r_even(sign_response.signature,params.full_order//2))
sign_response

In [ ]:
target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,sign_response.signature,data)

In [ ]:
keypair_dict = ECTesterTargetPCSC.encode_parameters(ParameterEnum.S, key)
keypair_dict.update(ECTesterTargetPCSC.encode_parameters(ParameterEnum.W, point_wrong))
print(target.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.KEYPAIR,
             keypair_dict))

In [ ]:
target.ecdsa_verify(KeypairEnum.KEYPAIR_LOCAL,SignatureEnum.ALG_ECDSA_SHA,sign_response.signature,data)